In [65]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
import scipy.stats as st
import datetime

from scipy.stats import linregress
from pprint import pprint

from config import weather_api_key
from config import geoapify_key

from citipy import citipy

In [100]:
#read in cxv file 
resort_city_data_df = pd.read_csv("Resources/resort_cities_current.csv", index_col="City_ID")

resort_city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,moengo,5.6167,-54.4000,30.56,62,77,2.59,SR,2023-04-28 10:40:29
1,ribeira grande,38.5167,-28.7000,17.65,67,40,9.77,PT,2023-04-28 10:40:29
2,hawaiian paradise park,19.5933,-154.9731,21.05,93,0,2.57,US,2023-04-28 10:40:29
3,yaren,-0.5472,166.9160,28.05,79,96,6.30,NR,2023-04-28 10:40:30
4,lihue,21.9789,-159.3672,20.97,89,100,5.14,US,2023-04-28 10:40:30
...,...,...,...,...,...,...,...,...,...
4374,tecuala,22.3986,-105.4569,31.52,29,19,3.14,MX,2023-04-28 11:05:42
4375,swellendam,-34.0226,20.4417,15.90,57,7,2.28,ZA,2023-04-28 11:05:42
4376,jalalabad,34.4265,70.4515,24.27,48,40,0.93,AF,2023-04-28 11:05:42


In [101]:
#set temperature perameters
resort_city_ideal_df = resort_city_data_df.loc[(resort_city_data_df["Max Temp"] < 30)
                                 & (resort_city_data_df["Max Temp"] > 24)
                                & (resort_city_data_df["Humidity"] < 70) 
                                & (resort_city_data_df["Humidity"] > 20)
                                & (resort_city_data_df["Wind Speed"] < 8)
                                & (resort_city_data_df["Cloudiness"] == 0)]
resort_city_ideal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
41,newman,37.3138,-121.0208,27.81,45,0,0.89,US,2023-04-28 10:40:44
48,laguna,38.4210,-121.4238,29.49,50,0,1.54,US,2023-04-28 10:40:45
60,los altos,37.3852,-122.1141,28.10,64,0,3.60,US,2023-04-28 10:38:03
88,khawr fakkan,25.3313,56.3420,29.03,53,0,4.34,AE,2023-04-28 10:40:55
120,umm qasr,30.0405,47.9168,29.93,24,0,3.82,IQ,2023-04-28 10:41:04


In [102]:
#count number of cities returned
resort_city_ideal_df.count()

City          186
Lat           186
Lng           186
Max Temp      186
Humidity      186
Cloudiness    186
Wind Speed    186
Country       186
Date          186
dtype: int64

In [103]:
#add columns for geoapi call info to go into 
resort_city_ideal_df["Number of Hotels"] = ""
resort_city_ideal_df["Number of Resturants"] = ""
resort_city_ideal_df["Bodies of Water"] = ""
resort_city_ideal_df["Tourist Attractions"] = ""
resort_city_ideal_df["Natural Places"] = ""
resort_city_ideal_df["Hospital"] = ""
resort_city_ideal_df["Entertainment"] = ""
resort_city_ideal_df["Rental Car"] = ""
resort_city_ideal_df["Airport"] = ""

resort_city_ideal_df.head()

C:\Users\sarah\AppData\Local\Temp\ipykernel_24432\390181530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resort_city_ideal_df["Number of Hotels"] = ""
C:\Users\sarah\AppData\Local\Temp\ipykernel_24432\390181530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resort_city_ideal_df["Number of Resturants"] = ""
C:\Users\sarah\AppData\Local\Temp\ipykernel_24432\390181530.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Number of Hotels,Number of Resturants,Bodies of Water,Tourist Attractions,Natural Places,Hospital,Entertainment,Rental Car,Airport
City_ID,,,,,,,,,,,,,,,,,,
41,newman,37.3138,-121.0208,27.81,45,0,0.89,US,2023-04-28 10:40:44,,,,,,,,,
48,laguna,38.4210,-121.4238,29.49,50,0,1.54,US,2023-04-28 10:40:45,,,,,,,,,
60,los altos,37.3852,-122.1141,28.10,64,0,3.60,US,2023-04-28 10:38:03,,,,,,,,,
88,khawr fakkan,25.3313,56.3420,29.03,53,0,4.34,AE,2023-04-28 10:40:55,,,,,,,,,
120,umm qasr,30.0405,47.9168,29.93,24,0,3.82,IQ,2023-04-28 10:41:04,,,,,,,,,


In [104]:
#build function for making api calls on different categories
def city_categories(categories, column, df):
    
    for index, row in df.iterrows():
        # get latitude, longitude from the DataFrame
        latitude = row["Lat"]
        longitude = row["Lng"]

        # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary

        # Set base URL
       
        radius = 16000

        # Set the parameters for the type of search
        filters = f"circle:{longitude},{latitude},{radius}"
        bias = f"proximity:{longitude},{latitude}"
        limit = 200

        # set up a parameters dictionary
        params = {
            "categories":categories,
            "limit":limit,
            "filter":filters,
            "bias":bias,
            "apiKey":geoapify_key    
        }

        # Set base URL
        base_url = "https://api.geoapify.com/v2/places"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        response_json = response.json()

        #pprint(response_json)


        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            df.loc[index, column] = len(response_json["features"])
        except (KeyError, IndexError):
            df.loc[index, column] = 0   
        
    return df 



In [105]:
#find cities with airports
categories = "airport.international"
column = "Airport"
resort_city_ideal_df = city_categories(categories, column, resort_city_ideal_df)

In [106]:
#drop cities with no airports
cities_airports = resort_city_ideal_df[(resort_city_ideal_df[["Airport"]] != 0).all(axis=1)]

In [107]:
#find hotels and resorts in each city
categories = "accommodation, beach.beach_resort"
column = "Number of Hotels"
ideal_cities_all_df = city_categories(categories, column, cities_airports)

In [108]:
#limit the number of hotels we want in each location 
resort_city_sorted = ideal_cities_all_df.sort_values("Number of Hotels")

cities_airports_hotel_lim = resort_city_sorted[(resort_city_sorted["Number of Hotels"] <= 25)]

In [109]:
#print dataframe
cities_airports_hotel_lim

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Number of Hotels,Number of Resturants,Bodies of Water,Tourist Attractions,Natural Places,Hospital,Entertainment,Rental Car,Airport
City_ID,,,,,,,,,,,,,,,,,,
4053,khark,29.2614,50.3306,26.32,54,0,2.94,IR,2023-04-28 11:03:50,0,,,,,,,,1
41,newman,37.3138,-121.0208,27.81,45,0,0.89,US,2023-04-28 10:40:44,1,,,,,,,,1
251,dogonbadan,30.3586,50.7981,24.05,29,0,2.06,IR,2023-04-28 10:41:52,1,,,,,,,,1
429,al wajh,26.2455,36.4525,25.97,65,0,4.63,SA,2023-04-28 10:42:53,1,,,,,,,,1
733,al awjam,26.5632,49.9433,28.06,34,0,3.60,SA,2023-04-28 10:44:40,2,,,,,,,,1
2604,santana do paraiso,-19.3636,-42.5686,28.18,51,0,2.57,BR,2023-04-28 10:55:45,3,,,,,,,,1
2329,brownwood,31.7093,-98.9912,25.00,57,0,7.20,US,2023-04-28 10:54:21,3,,,,,,,,1
608,al kharijah,25.4514,30.5464,25.47,23,0,5.55,EG,2023-04-28 10:43:53,3,,,,,,,,1
3633,goya,-29.1400,-59.2626,28.96,31,0,4.87,AR,2023-04-28 11:01:27,4,,,,,,,,3


In [124]:
#find bodies of water in locations 
categories = "natural.water.sea,natural.water.reef,natural.water"
column = "Bodies of Water"
ideal_cities_all_df = city_categories(categories, column, cities_airports_hotel_lim)

In [111]:
#find tourist attractions present in cities 
categories = "tourism"
column = "Tourist Attractions"
ideal_cities_all_df = city_categories(categories, column, cities_airports_hotel_lim)

In [112]:
#find resturants present locations 
categories = "catering.restaurant"
column = "Number of Resturants"
ideal_cities_all_df = city_categories(categories, column, cities_airports_hotel_lim)

In [113]:
#find natural areas in locations 
categories = "natural"
column = "Natural Places"
ideal_cities_all_df = city_categories(categories, column, cities_airports_hotel_lim)

In [114]:
#find hospitals in locations 
categories = "healthcare.hospital"
column = "Hospital"
ideal_cities_all_df = city_categories(categories, column, cities_airports_hotel_lim)

In [115]:
#find rental car agencies in locations 
categories = "rental.car"
column = "Rental Car"
ideal_cities_all_df = city_categories(categories, column, cities_airports_hotel_lim)

In [116]:
#find entertainment opportunties in locations 
categories = "entertainment"
column = "Entertainment"
ideal_cities_all_df = city_categories(categories, column, cities_airports_hotel_lim)

In [117]:
#print df
cities_airports_hotel_lim

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Number of Hotels,Number of Resturants,Bodies of Water,Tourist Attractions,Natural Places,Hospital,Entertainment,Rental Car,Airport
City_ID,,,,,,,,,,,,,,,,,,
4053,khark,29.2614,50.3306,26.32,54,0,2.94,IR,2023-04-28 11:03:50,0,0,5,3,7,1,0,0,1
41,newman,37.3138,-121.0208,27.81,45,0,0.89,US,2023-04-28 10:40:44,1,14,200,3,200,0,3,0,1
251,dogonbadan,30.3586,50.7981,24.05,29,0,2.06,IR,2023-04-28 10:41:52,1,3,2,0,13,4,1,0,1
429,al wajh,26.2455,36.4525,25.97,65,0,4.63,SA,2023-04-28 10:42:53,1,0,2,3,1,1,0,0,1
733,al awjam,26.5632,49.9433,28.06,34,0,3.60,SA,2023-04-28 10:44:40,2,45,28,9,36,10,0,0,1
2604,santana do paraiso,-19.3636,-42.5686,28.18,51,0,2.57,BR,2023-04-28 10:55:45,3,17,48,5,106,10,1,0,1
2329,brownwood,31.7093,-98.9912,25.00,57,0,7.20,US,2023-04-28 10:54:21,3,6,21,0,22,1,1,0,1
608,al kharijah,25.4514,30.5464,25.47,23,0,5.55,EG,2023-04-28 10:43:53,3,2,24,8,26,1,0,0,1
3633,goya,-29.1400,-59.2626,28.96,31,0,4.87,AR,2023-04-28 11:01:27,4,0,200,3,200,4,2,0,3


In [122]:
#sort by hotels 
cities_sorted_by_hotels = cities_airports.sort_values("Number of Hotels", ascending=True)

cities_sorted_by_hotels

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Number of Hotels,Number of Resturants,Bodies of Water,Tourist Attractions,Natural Places,Hospital,Entertainment,Rental Car,Airport
City_ID,,,,,,,,,,,,,,,,,,
4053,khark,29.2614,50.3306,26.32,54,0,2.94,IR,2023-04-28 11:03:50,0,,,,,,,,1
41,newman,37.3138,-121.0208,27.81,45,0,0.89,US,2023-04-28 10:40:44,1,,,,,,,,1
251,dogonbadan,30.3586,50.7981,24.05,29,0,2.06,IR,2023-04-28 10:41:52,1,,,,,,,,1
429,al wajh,26.2455,36.4525,25.97,65,0,4.63,SA,2023-04-28 10:42:53,1,,,,,,,,1
733,al awjam,26.5632,49.9433,28.06,34,0,3.60,SA,2023-04-28 10:44:40,2,,,,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,caxias,38.7031,-9.2767,26.90,47,0,6.17,PT,2023-04-28 10:57:30,200,,,,,,,,2
4018,dubai,25.2582,55.3047,27.14,61,0,2.57,AE,2023-04-28 11:01:57,200,,,,,,,,1
2473,paysandu,-32.3214,-58.0756,27.00,38,0,6.61,UY,2023-04-28 10:52:41,200,,,,,,,,1


In [123]:
#eport to csv 
cities_airports_hotel_lim.to_csv("Resources/cities_airports_hotel_lim.csv", index_label="City_ID")